In [2]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨.csv')
customer = pd.read_csv('해운대 입장객수.csv')

In [ ]:
weather.head(5)

,날짜,강수_관측값,기온,습도,체감온도,평균풍속,평균기압,평균수온,평균최대파고,평균파주기
0,2012-05-01,10.5,20.4,62.1,21.5,6.5,1009.0,15.2,1.2,4.8
1,2012-05-02,1.0,19.6,70.5,21.4,7.8,1004.0,15.5,2.1,5.7
2,2012-05-03,0.1,20.7,77.5,22.9,4.1,1004.0,16.4,1.9,6.6
3,2012-05-04,0.0,19.9,70.7,21.7,4.1,1006.5,16.4,1.6,7.5
4,2012-05-05,0.0,24.8,47.2,24.7,6.1,1004.9,15.8,1.9,7.8


In [ ]:
customer.head(5)

,방문일,방문객수(명)
0,2018-06-01,"28,423"
1,2018-06-02,"29,540"
2,2018-06-03,"28,603"
3,2018-06-04,"27,336"
4,2018-06-05,"37,173"


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수(명)'] = customer['방문객수(명)'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1618 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1618 non-null   datetime64[ns]
 1   강수_관측값  1618 non-null   float64       
 2   기온      1618 non-null   float64       
 3   습도      1618 non-null   float64       
 4   체감온도    1618 non-null   float64       
 5   평균풍속    1567 non-null   float64       
 6   평균기압    1557 non-null   float64       
 7   평균수온    1511 non-null   float64       
 8   평균최대파고  1552 non-null   float64       
 9   평균파주기   1551 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 126.5 KB


In [ ]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   방문일      332 non-null    datetime64[ns]
 1   방문객수(명)  332 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.3 KB


In [ ]:
# merge data : 방문객수 예측을 위한 데이터
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수(명)"]]

# weather data : 날씨 예측을 위한 데이터(na는 제거하는 방향으로 일단 진행)
weather = weather.dropna()

## 방문객 수 예측

In [ ]:
# 방문객 수 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(total_data.iloc[:, :-1], total_data['방문객수(명)'], test_size=0.2)

In [ ]:
# 방문객 수 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
# 방문객 수 예측을 위한  model training
model=Sequential()
model.add(Dense(256,input_dim=9, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 256)               2560      
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 1)               

In [ ]:
# 학습 진행
earlyStopping=EarlyStopping(monitor='val_loss',patience=10,mode='min',verbose=1,restore_best_weights=True) 
model.compile(loss='mse',optimizer='adam')
hist=model.fit(train_x_scaled,train_y,epochs=500,batch_size=16,validation_split=0.1, callbacks=[earlyStopping])

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 입장객 수: ', pred_y[-1:])

2/2 [==============================] - 0s 11ms/step - loss: 3316367872.0000
loss:  3316367872.0
예상 입장객 수:  [[61907.344]]


## 강수_관측값 예측

In [ ]:
# 사용 데이터
weather.info()

# input data
weather_input = weather[["기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = weather[['강수_관측값']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1486 non-null   datetime64[ns]
 1   강수_관측값  1486 non-null   float64       
 2   기온      1486 non-null   float64       
 3   습도      1486 non-null   float64       
 4   체감온도    1486 non-null   float64       
 5   평균풍속    1486 non-null   float64       
 6   평균기압    1486 non-null   float64       
 7   평균수온    1486 non-null   float64       
 8   평균최대파고  1486 non-null   float64       
 9   평균파주기   1486 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 127.7 KB


In [ ]:
# 강수_관측값 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input.values, weather_output.values, test_size=0.2)

In [ ]:
# 강수_관측값 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
# 강수_관측값 예측을 위한 data reshape
train_x_scaled = train_x_scaled.reshape(-1, 8, 1)
test_x_scaled = test_x_scaled.reshape(-1, 8, 1)

In [ ]:
# 강수_관측값 예측을 위한 lstm 학습
model=Sequential()
model.add(LSTM(units=128,input_shape=(8,1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_55 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 65        
Total params: 74,881
Trainable params: 74,881
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 학습 진행
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1,restore_best_weights=True)
model.compile(loss='mse', optimizer='adam')
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=16, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

60/60 - 1s - loss: 371.6552 - val_loss: 318.3376
Epoch 103/500
60/60 - 1s - loss: 379.6888 - val_loss: 291.2632
Epoch 104/500
60/60 - 1s - loss: 378.7491 - val_loss: 289.9620
Epoch 105/500
60/60 - 1s - loss: 369.7699 - val_loss: 298.5469
Epoch 106/500
60/60 - 2s - loss: 372.6702 - val_loss: 296.6628
Epoch 107/500
60/60 - 2s - loss: 367.3729 - val_loss: 278.9319
Epoch 108/500
60/60 - 2s - loss: 386.0328 - val_loss: 312.4142
Epoch 109/500
60/60 - 2s - loss: 393.4918 - val_loss: 300.4136
Epoch 110/500
60/60 - 2s - loss: 371.3260 - val_loss: 289.0213
Epoch 111/500
60/60 - 2s - loss: 378.1547 - val_loss: 286.9359
Epoch 112/500
60/60 - 2s - loss: 394.7773 - val_loss: 294.9448
Epoch 113/500
60/60 - 2s - loss: 385.0535 - val_loss: 287.0111
Epoch 114/500
60/60 - 2s - loss: 377.9507 - val_loss: 287.1523
Epoch 115/500
60/60 - 2s - loss: 374.5316 - val_loss: 302.8996
Epoch 116/500
60/60 - 2s - loss: 372.5007 - val_loss: 295.0930
Epoch 117/500
60/60 - 2s - loss: 367.9634 - val_loss: 291.2569
Restor

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 강수_관측값 : ', pred_y[-1:])

10/10 [==============================] - 0s 4ms/step - loss: 216.2467
loss:  216.2467041015625
예상 강수_관측값 :  [[55.593212]]


## 기온 예측

In [ ]:
# 사용 데이터
weather.info()

# input data
weather_input = weather[["강수_관측값", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = weather[['기온']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1486 non-null   datetime64[ns]
 1   강수_관측값  1486 non-null   float64       
 2   기온      1486 non-null   float64       
 3   습도      1486 non-null   float64       
 4   체감온도    1486 non-null   float64       
 5   평균풍속    1486 non-null   float64       
 6   평균기압    1486 non-null   float64       
 7   평균수온    1486 non-null   float64       
 8   평균최대파고  1486 non-null   float64       
 9   평균파주기   1486 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 127.7 KB


In [ ]:
# 기온 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input.values, weather_output.values, test_size=0.2)

In [ ]:
# 기온 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
# 기온 예측을 위한 data reshape
train_x_scaled = train_x_scaled.reshape(-1, 8, 1)
test_x_scaled = test_x_scaled.reshape(-1, 8, 1)

In [ ]:
# 기온 예측을 위한 lstm 학습
model=Sequential()
model.add(LSTM(units=128,input_shape=(8,1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_57 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_58 (Dense)             (None, 1)                 65        
Total params: 74,881
Trainable params: 74,881
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 학습 진행
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1,restore_best_weights=True)
model.compile(loss='mse', optimizer='adam')
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=16, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

Epoch 1/500
60/60 - 6s - loss: 294.8029 - val_loss: 16.9127
Epoch 2/500
60/60 - 2s - loss: 14.5535 - val_loss: 14.3196
Epoch 3/500
60/60 - 2s - loss: 14.2070 - val_loss: 14.2903
Epoch 4/500
60/60 - 2s - loss: 14.2028 - val_loss: 14.9367
Epoch 5/500
60/60 - 2s - loss: 14.2439 - val_loss: 14.2816
Epoch 6/500
60/60 - 2s - loss: 14.4665 - val_loss: 14.3856
Epoch 7/500
60/60 - 2s - loss: 14.2717 - val_loss: 14.2550
Epoch 8/500
60/60 - 1s - loss: 14.1948 - val_loss: 14.6336
Epoch 9/500
60/60 - 1s - loss: 14.6506 - val_loss: 14.8402
Epoch 10/500
60/60 - 1s - loss: 14.4735 - val_loss: 14.4695
Epoch 11/500
60/60 - 1s - loss: 14.1548 - val_loss: 14.2540
Epoch 12/500
60/60 - 1s - loss: 14.2216 - val_loss: 14.6555
Epoch 13/500
60/60 - 1s - loss: 14.4435 - val_loss: 15.0292
Epoch 14/500
60/60 - 1s - loss: 14.4140 - val_loss: 14.2505
Epoch 15/500
60/60 - 2s - loss: 14.2542 - val_loss: 14.8203
Epoch 16/500
60/60 - 1s - loss: 14.3089 - val_loss: 14.4372
Epoch 17/500
60/60 - 1s - loss: 14.2792 - val_lo

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 기온 예측값 : ', pred_y[-1:])

10/10 [==============================] - 0s 4ms/step - loss: 12.3377
loss:  12.337726593017578
예상 기온 예측값 :  [[26.389303]]
